### Recommend Anime dựa trên hệ thống hybrid (collaborative và content-based filtering)
    - Kết hợp kết quả dự đoán cho kết quả từ ALS đã tìm được và kết quả dự đoán từ embedding (content-based filtering)
    - Sử dụng phương pháp lai để loại trừ khuyết điểm của các mô hình

In [1]:
import pandas as pd
import numpy as np
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import metrics_eval
import matplotlib.pyplot as plt

import faiss
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
from collections import defaultdict

import nltk
from nltk.stem import PorterStemmer

from functools import *
from sklearn.feature_extraction.text import CountVectorizer

c:\Users\LU QUOC PHAP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_rating = pd.read_csv("Dataset/train_rating.csv")
test_rating = pd.read_csv("Dataset/test_rating.csv")
embedding_anime = pd.read_csv("Dataset/embedding_anime.csv")
rating = pd.read_csv("Dataset/rating.csv")

In [3]:
# 1. Tiền xử lý text (từ item-based)
ps = PorterStemmer()
def stems(text: str):
    l = list(map(ps.stem, text.split()))
    return " ".join(l)

embedding_anime['tags'] = embedding_anime['description'].apply(stems)

# 2. Vector hóa (từ item-based)
cv = CountVectorizer(max_features=1000, stop_words='english')
vector = cv.fit_transform(embedding_anime['tags']).toarray()

# 3. Chuẩn hóa vector và tạo index FAISS (từ item-based)
vector = vector.astype(np.float32)
row_norm = np.linalg.norm(vector, axis=1, keepdims=True) + 1e-12
vector = vector / row_norm

item_ids = embedding_anime['anime_id'].to_numpy()
row_of = {int(a): i for i, a in enumerate(item_ids)}
n_items, dim = vector.shape

topk_neighbors = 200  
index = faiss.IndexFlatIP(dim)
index.add(vector.astype('float32'))
sim_all, idx_all = index.search(vector.astype('float32'), topk_neighbors + 1)

nbr_idx = idx_all[:, 1:].astype(np.int32)
nbr_sim = sim_all[:, 1:].astype(np.float32)

# 4. Tạo User Likes Map (từ item-based)
liked_threshold = 5          
limit_liked_per_user = 30    
rank_decay = 0.9             
k_recommend = 20  

df_likes = (train_rating[
        (train_rating['rating'] >= liked_threshold) | (train_rating['rating'] == -1)
    ]
            .sort_values(['user_id', 'rating'], ascending=[True, False]))
user_likes_map = df_likes.groupby('user_id')['anime_id'].apply(list).to_dict()

item_popularity = train_rating['anime_id'].value_counts()
cold_start_top = [int(a) for a in item_popularity.index if int(a) in row_of][:k_recommend]

print(f"Item-Based: Sẵn sàng (n_items={n_items})")

Item-Based: Sẵn sàng (n_items=12017)


In [4]:
# 1. Ánh xạ Confidence (từ ALS)
def confidence_mapping(r):
    if r == -1: return 5
    elif r >= 8: return 10
    elif r >= 5: return 4
    else: return 1

rating['confidence'] = rating['rating'].apply(confidence_mapping)
train_rating['confidence'] = train_rating['rating'].apply(confidence_mapping)
test_rating['confidence'] = test_rating['rating'].apply(confidence_mapping)

# 2. Tạo User/Item Maps và Ma trận (từ ALS)
all_users = sorted(rating["user_id"].unique())
all_items = sorted(rating["anime_id"].unique()) # Đây là các item trong mô hình ALS

user_map = {u: i for i, u in enumerate(all_users)}
item_map = {it: j for j, it in enumerate(all_items)}

# *** PHẦN MỚI QUAN TRỌNG ***
# Tạo map ngược từ index của ALS về anime_id
idx_to_anime_id_map = {v: k for k, v in item_map.items()}


train_rating["user_idx"] = train_rating["user_id"].map(user_map)
train_rating["item_idx"] = train_rating["anime_id"].map(item_map)
test_rating["user_idx"]  = test_rating["user_id"].map(user_map)
test_rating["item_idx"]  = test_rating["anime_id"].map(item_map)

user_items = csr_matrix(
    (train_rating["confidence"].values, (train_rating["user_idx"].values, train_rating["item_idx"].values)),
    shape=(len(all_users), len(all_items)),
    dtype=np.float32
)

print(f"ALS: Sẵn sàng (Users={len(all_users)}, Items={len(all_items)})")

ALS: Sẵn sàng (Users=73515, Items=11200)


In [5]:
# Huấn luyện mô hình ALS
model = AlternatingLeastSquares(factors=150, regularization=0.1, iterations=100, random_state=42)
model.fit(user_items)

print("ALS Model trained.")

c:\Users\LU QUOC PHAP\AppData\Local\Programs\Python\Python312\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 100/100 [04:50<00:00,  2.90s/it]

ALS Model trained.


In [6]:
# Hàm chuẩn hóa điểm về [0, 1]
scaler = MinMaxScaler()
def normalize_scores(scores):
    if not isinstance(scores, np.ndarray):
        scores = np.array(scores)
    
    # Nếu tất cả điểm bằng nhau (ví dụ: [0, 0, 0]), scaler sẽ lỗi
    if scores.max() == scores.min():
        return np.zeros_like(scores)
        
    return scaler.fit_transform(scores.reshape(-1, 1)).flatten()

# *** PHẦN ĐIỀU CHỈNH ***
def get_item_based_scores(user_id: int,
                            rank_decay_val: float = rank_decay,
                            l_cap: int = limit_liked_per_user):
    
    liked_ids = user_likes_map.get(int(user_id), [])
    if not liked_ids:
        # Trả về 0 cho mọi item nếu là cold start
        return np.zeros(n_items, dtype=np.float32)

    if l_cap is not None and len(liked_ids) > l_cap:
        liked_ids = liked_ids[:l_cap]

    liked_rows = np.fromiter((row_of[a] for a in liked_ids if a in row_of), dtype=np.int32, count=-1)
    if liked_rows.size == 0:
        return np.zeros(n_items, dtype=np.float32)

    cand_idx = nbr_idx[liked_rows]    # (L, K)
    cand_sim = nbr_sim[liked_rows]    # (L, K)

    if rank_decay_val != 1.0:
        ranks = np.arange(cand_idx.shape[1], dtype=np.float32)  # 0..K-1
        cand_sim = cand_sim * (rank_decay_val ** ranks)[None, :]

    # cộng dồn vector hoá
    scores = np.zeros(n_items, dtype=np.float32)
    np.add.at(scores, cand_idx.ravel(), cand_sim.ravel())

    # loại item đã xem (chúng ta sẽ làm điều này ở hàm hybrid cuối cùng)
    # scores[liked_rows] = -np.inf 
    
    return scores

In [7]:
def recommend_hybrid(user_id, k=20, n_candidates=100, lambda_val=0.6):
    
    # 0. Kiểm tra Cold Start
    if user_id not in user_map or user_id not in user_likes_map:
        return cold_start_top[:k] # Trả về popular cho user mới
    
    user_idx = user_map[user_id]
    
    # 1. Lấy Ứng viên từ ALS
    # (Hàm này trả về item_idx của ALS)
    als_indices, als_scores = model.recommend(user_idx, user_items[user_idx], N=n_candidates)
    
    # 2. Lấy *tất cả* điểm từ Item-Based (IB)
    # (Hàm này trả về điểm cho item_row của IB)
    ib_scores_all = get_item_based_scores(user_id)
    
    # Chuẩn hóa điểm số
    als_scores_norm = normalize_scores(als_scores)
    # Chỉ chuẩn hóa các điểm IB của các ứng viên ALS (tránh chuẩn hóa cả 12017 item)
    ib_candidate_scores = []
    
    
    hybrid_candidates = []
    
    # Lấy các item người dùng đã xem để lọc
    liked_items_set = set(user_likes_map.get(user_id, []))
    
    for i in range(len(als_indices)):
        item_idx_als = als_indices[i]
        
        # Chuyển từ index của ALS -> anime_id
        anime_id = idx_to_anime_id_map.get(item_idx_als)
        
        if anime_id is None or anime_id in liked_items_set:
            continue
            
        # Chuyển từ anime_id -> index của Item-Based (row_of)
        if anime_id not in row_of:
            continue # Anime này có trong ALS nhưng không có trong model Item-Based
            
        item_row_ib = row_of[anime_id]
        
        # Lấy điểm
        als_score = als_scores_norm[i]
        ib_score = ib_scores_all[item_row_ib] # Lấy điểm IB tương ứng
        
        hybrid_candidates.append({
            "anime_id": anime_id,
            "als_score_norm": als_score,
            "ib_score_raw": ib_score  # Sẽ chuẩn hóa sau
        })

    if not hybrid_candidates:
        return cold_start_top[:k] # Không tìm thấy ứng viên nào
        
    # Chuẩn hóa riêng điểm IB của các ứng viên
    ib_scores_to_norm = [c['ib_score_raw'] for c in hybrid_candidates]
    ib_scores_norm = normalize_scores(ib_scores_to_norm)
    
    # 3. Tính điểm Hybrid
    final_scores = []
    for i, c in enumerate(hybrid_candidates):
        ib_score = ib_scores_norm[i]
        als_score = c['als_score_norm']
        
        final_score = (lambda_val * als_score) + ((1 - lambda_val) * ib_score)
        
        final_scores.append((c['anime_id'], final_score))
        
    # 4. Sắp xếp và trả về Top K
    final_scores.sort(key=lambda x: x[1], reverse=True)
    
    return [anime_id for anime_id, score in final_scores[:k]]

In [8]:
# Thử nghiệm trên một user
test_user_id = 9229
print(f"--- Đề xuất cho User {test_user_id} ---")

hybrid_recs_ids = recommend_hybrid(test_user_id, k=20, lambda_val=0.7)

# Lấy tên anime
hybrid_recs_names = embedding_anime[embedding_anime['anime_id'].isin(hybrid_recs_ids)]['name']
print(hybrid_recs_names.to_list())

--- Đề xuất cho User 9229 ---
['Baccano!', 'Rose of Versailles', 'Detective Conan Movie 13: The Raven Chaser', 'Mirai Shounen Conan', 'Clannad: Mou Hitotsu no Sekai, Tomoyo-hen', 'Detective Conan OVA 09: The Stranger in 10 Years...', 'Takarajima', 'Akachan to Boku', 'Clannad: After Story - Mou Hitotsu no Sekai, Kyou-hen', 'Detective Conan Movie 01: The Timed Skyscraper', 'Lupin III vs. Detective Conan', 'Watashi no Ashinaga Ojisan', 'Itazura na Kiss', 'Ai no Wakakusa Monogatari', 'Detective Conan OVA 01: Conan vs. Kid vs. Yaiba', 'Digimon Adventure 02', 'Kazoku Robinson Hyouryuuki: Fushigi na Shima no Flone', 'Alps no Shoujo Heidi', 'Digimon Frontier', 'Jungle Book Shounen Mowgli']


In [9]:
# Đánh giá trên toàn bộ tập Test

# 1. Lấy Ground Truth (từ 2 notebook)
ground_truth = (
    test_rating
    .groupby('user_id')['anime_id']
    .apply(set)
    .to_dict()
)

# Lọc ground_truth để chỉ giữ các user có trong ALS (để so sánh công bằng)
all_users_test = [u for u in ground_truth.keys() if u in user_map]
ground_truth_filtered = {u: ground_truth[u] for u in all_users_test}

print(f"Bắt đầu đánh giá trên {len(all_users_test)} users...")

# 2. Chạy dự đoán (dùng song song)
def _predict_one_hybrid(u):
    return int(u), recommend_hybrid(int(u), k=20, lambda_val=0.7) # Giữ lambda = 0.6

# Giảm batch_size nếu bị treo/hết bộ nhớ
predicted_list_hybrid = Parallel(n_jobs=-1, backend="loky", batch_size=128, verbose=5)(
    delayed(_predict_one_hybrid)(u) for u in all_users_test
)

predicted_hybrid = dict(predicted_list_hybrid)

# 3. Tính toán Metrics
result_hybrid = metrics_eval.evaluate_all(predicted_hybrid, ground_truth_filtered, k=15)

print("\n--- Kết quả Đánh giá Mô hình Hybrid (λ=0.6) ---")
print(result_hybrid)

Bắt đầu đánh giá trên 71102 users...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done 6168 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 17688 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 33816 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 54552 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 70896 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 71079 out of 71102 | elapsed:  8.3min remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 71102 out of 71102 | elapsed:  8.4min finished



--- Kết quả Đánh giá Mô hình Hybrid (λ=0.6) ---
{'Precision@15': 0.2938529624108089, 'Recall@15': 0.3081397886460829, 'MAP@15': 0.27814608864763496}


- đã chạy (8)